In [23]:
import subprocess

plasmids_file = open("../../fex_117_sample_sheet.tsv")
header = plasmids_file.readline()

def get_plasmid_length(fl):
    pl_fl = open(fl)
    header = pl_fl.readline()
    total = ""
    for line in pl_fl:
        total += line.strip()
    piggy = "CCCTAGAAAGATAATCATATTGTGACGT" in total.upper()
    lenti = "CTGGGAGCTCTCTGGCTAACTAGGGAA" in total.upper()
    ptype = "NONE"
    if piggy:
        ptype = "PIGGY"
    elif lenti:
        ptype = "LENTI"
    return((len(total),piggy,lenti,ptype))

def mummer_repeat_sizes(plasmid):
    command_array = ["/analysis/2021_08_26_PlasmidSeq_paper/repeats/MUMmer3.23/repeat-match","-E",plasmid]
    # print(" ".join(command_array))
    process = subprocess.run(command_array,capture_output=True,text=True)
    lines = process.stdout.split("\n")
    size = 0
    count = 0
    largest = 0
    
    for line in lines[2:len(lines)]:
        if len(line.split()) < 3:
            # print("LN:" + str(line))
            count += 0
        else:
            sz = int(line.split()[2])
            size += sz
            count += 1
            if sz > largest:
                largest = sz
    return((size,count,largest))

output = open("2021_12_13_repeat_content.txt","w")
output.write("sample\tname\tlength\tpiggyBac\tlentiVirus\tptype\trepeatTotal\trepeatCount\trepeatLargest\tunrepeatProp\n")
for line in plasmids_file:
    plasmid_map  = "../../" + line.strip().split("\t")[2]
    sample  = line.strip().split("\t")[0]
    sample_name  = line.strip().split("\t")[1]
    length  = get_plasmid_length(plasmid_map)
    repeats = mummer_repeat_sizes(plasmid_map)
    output.write(sample + "\t" + sample_name + "\t" + str(length[0]) + "\t" + str(length[1]) + "\t" + str(length[2]) + "\t" + str(length[3]) + "\t" + str(repeats[0]) + "\t" + str(repeats[1]) + "\t" + str(repeats[2]) + "\t" + str((length[0]-repeats[0])/length[0]) + "\n")
output.close()